In [13]:
import os
# from os import environ
# environ["KERAS_BACKEND"] = "plaidml.keras.backend"
# import tifffile as tiff
# import re
from sklearn.utils import shuffle
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten
from keras.optimizers import Adam
# from tensorflowkeras.optimizers.schedules import ExponentialDecay
from keras.callbacks import EarlyStopping

In [27]:
#input training labels (file and label)
train = pd.read_csv('train_labels.csv')
#putting labels as string for image generator
train['label'] = train['label'].astype(str)
#changing labels of files to match filesystem in google colab
train['id'] += '.tif'
# train['id'] = train['id'].apply(lambda x: 'train/' + x)
#creating random state to apply shuffling to data
train = shuffle(train, random_state = 42)


#split this into training and validation for model
train, valid = train_test_split(train, test_size = 0.25)

#sample_submission
sample = pd.read_csv('sample_submission.csv')

#creating test data frame to import data, creates id col with name of file
test = pd.DataFrame({'id':os.listdir('test')})

#setting batch size for entire project
batch_size = 200


In [15]:
#creating method to generate training and validation images
#creates validation set of 0.2 size of training data and scales pixels
generator_data = ImageDataGenerator(
    rescale=1./255.,
    validation_split=0.2
)


#getting image generator based upon labels in train df
train_generator = generator_data.flow_from_dataframe(
    dataframe = train,
    directory = 'train',
    x_col = 'id',
    y_col = 'label',
    shuffle = True,
    featurewise_std_normalization = True,
    class_mode = 'binary',
    batch_size = batch_size,
    random_state = 42,
    horizontal_flip = True,
    target_size = (96,96)
)

#getting image generator based upon lables in valid df
valid_generator = generator_data.flow_from_dataframe(
    dataframe = valid,
    directory = 'train',
    x_col = 'id',
    y_col = 'label', 
    shuffle = True,
    featurewise_std_normalization = True,
    class_mode = 'binary',
    batch_size = batch_size,
    random_state = 42,
    horizontal_flip = True,
    target_size = (96,96)
)

Found 165018 validated image filenames belonging to 2 classes.
Found 55007 validated image filenames belonging to 2 classes.


In [18]:
#beginning model

#building model with layers
model = Sequential([
    Conv2D(16, kernel_size = 3, activation = 'relu', input_shape = (96,96,3)),
    BatchNormalization(),
    Conv2D(16, kernel_size = 3, activation = 'relu', strides = (2,2)),
    BatchNormalization(),
    MaxPooling2D(pool_size = (2,2)),
    Conv2D(64, kernel_size = 3, activation = 'relu', strides = (2,2)),
    BatchNormalization(),
    Conv2D(64, kernel_size = 3, activation = 'relu', strides = (2,2)),
    BatchNormalization(),
    MaxPooling2D(pool_size = (2,2)),
    Flatten(),
    Dense(8, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
]);

from keras.optimizers import SGD
opt = SGD(learning_rate = 0.001,
  nesterov = True,
  momentum = 0.99
)



#model compiler
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

#fitting the model
model.fit(
    train_generator,
    validation_data = valid_generator,
    validation_steps = 500,
    batch_size = batch_size,
    epochs = 10
)


Epoch 1/10


2023-02-12 22:33:53.443728: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


645/645 [==============================] - ETA: 0s - loss: 0.4194 - accuracy: 0.8107

2023-02-12 22:34:48.084304: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


645/645 [==============================] - 73s 113ms/step - loss: 0.4194 - accuracy: 0.8107 - val_loss: 0.3891 - val_accuracy: 0.8284
Epoch 2/10
645/645 [==============================] - 56s 87ms/step - loss: 0.3477 - accuracy: 0.8479
Epoch 3/10
645/645 [==============================] - 56s 87ms/step - loss: 0.3224 - accuracy: 0.8614
Epoch 4/10
645/645 [==============================] - 56s 87ms/step - loss: 0.2996 - accuracy: 0.8743
Epoch 5/10
645/645 [==============================] - 54s 84ms/step - loss: 0.2809 - accuracy: 0.8824
Epoch 6/10
645/645 [==============================] - 55s 85ms/step - loss: 0.2671 - accuracy: 0.8891
Epoch 7/10
645/645 [==============================] - 57s 88ms/step - loss: 0.2540 - accuracy: 0.8950
Epoch 8/10
645/645 [==============================] - 55s 85ms/step - loss: 0.2423 - accuracy: 0.9008
Epoch 9/10
645/645 [==============================] - 55s 85ms/step - loss: 0.2398 - accuracy: 0.9024
Epoch 10/10
645/645 [=============================

In [ ]:
data_generator_test = ImageDataGenerator(
    rescale = 1./255.,
)

#creating sequence of image generator
test_generator = data_generator_test.flow_from_dataframe(
    dataframe = test,
    directory ='test',
    x_col = 'id', 
    y_col = None,
    target_size = (96,96),         
    batch_size = 1,
    shuffle = False,
    class_mode = None
)


#prediction
predict = model.predict(
    x = test_generator
)

Found 57458 validated image filenames.


2023-02-12 22:52:39.406581: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


57458/57458 [==============================] - 124s 2ms/step
